In [15]:
def read_undet_indices(result_path):
    undet_indices = []
    with open(result_path,'r') as f:
        header_found = False
        for line in f:
            # Strip leading/trailing whitespace characters
            line = line.strip()
            
            # Check if the current line is the header
            if line == "# Undetected object indices":
                header_found = True
                continue
            
            # If the header is found, store the subsequent lines
            if header_found:
                undet_indices.append(line)
    return undet_indices

cgc_result_path = '/mnt/2tb-1/satellitepy/fineair_experiments/roi_trans_cgc/evaluations/mAP_values.txt'
fac_result_path = '/mnt/2tb-1/satellitepy/fineair_experiments/roi_trans_fac/evaluations/mAP_values.txt'
cgc_undet_indices = read_undet_indices(cgc_result_path)
fac_undet_indices = read_undet_indices(fac_result_path)
# Print the stored lines (optional)
# Find overlapping items (intersection)
overlapping_items = list(set(cgc_undet_indices) & set(fac_undet_indices))

# Find non-overlapping items in List 1
nonoverlapping_list1 = list(set(cgc_undet_indices) - set(fac_undet_indices))

# Find non-overlapping items in List 2
nonoverlapping_list2 = list(set(fac_undet_indices) - set(cgc_undet_indices))

# Print the results
print("Total # Undetected objects by the CGC model:", len(cgc_undet_indices))
print("Total # Undetected objects by the FAC model:", len(fac_undet_indices))
print("# Undetected objects by both:", len(overlapping_items))
print("# Undetected objects only by the CGC model:", len(nonoverlapping_list1))
print("# Undetected objects only by the FAC model:", len(nonoverlapping_list2))



Total # Undetected objects by the CGC model: 157
Total # Undetected objects by the FAC model: 504
# Undetected objects by both: 154
# Undetected objects only by the CGC model: 3
# Undetected objects only by the FAC model: 350
